## 模型融合

### 载入数据

In [1]:
import pandas

In [2]:
data = "data/Pima-Indians-Diabetes.csv"
#年纪、怀孕、血液检查的次数... 匹马印第安人糖尿病的数据集
df = pandas.read_csv(data)
df.columns = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']

In [3]:
df.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df['class'].unique()

array([1, 0])

### 1.投票器模型融合

In [5]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
import warnings

In [6]:
warnings.filterwarnings('ignore')

In [7]:
df.head(2)

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [8]:
array = df.values

In [9]:
array

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

In [10]:
X = array[:,0:8]
Y = array[:,8]
kfold = model_selection.KFold(n_splits=5, random_state=2018)

# 创建投票器的子模型
estimators = []

# 第1个模型，逻辑回归
model_1 = LogisticRegression()
estimators.append(('logistic', model_1))

# 第2个模型，决策树分类器
model_2 = DecisionTreeClassifier()
estimators.append(('dt', model_2))

# 第3个模型，支持向量机分类器
model_3 = SVC()
estimators.append(('svm', model_3))

# 构建投票器
ensemble = VotingClassifier(estimators)

# 对投票器分类器，做交叉验证，拿到5个(fold上面的)得分
result = model_selection.cross_val_score(ensemble, X, Y, cv=kfold)

# 对5个得分求平均，代表在数据集上的效果
print(result.mean())

0.7462015109073933


### Bagging

In [11]:
from sklearn.ensemble import BaggingClassifier

In [12]:
# Bagging要求子模型都是同类的，比如说这里都是决策树
dt = DecisionTreeClassifier()
# 用100颗决策树去做集成
num = 100
# 5折交叉验证的切分方法对象
kfold = model_selection.KFold(n_splits=5, random_state=2018)
# 用100颗决策树去构建一个bagging的集成模型
model = BaggingClassifier(base_estimator=dt, n_estimators=num, random_state=2018)
# 对Bagging分类器，做交叉验证，拿到5个(fold上面的)得分
result = model_selection.cross_val_score(model, X, Y, cv=kfold)
# 对5个得分求平均，代表在数据集上的效果
print(result.mean())

0.7656480774127834


### RandomForest

In [13]:
# 就是用决策树去集成的模型
# 但是构建每颗树的时候，会对样本和特征去做采样
from sklearn.ensemble import RandomForestClassifier

# 树的棵树
num_trees = 100

# 每棵树最多用多少特征
max_feature_num = 5

# 5折交叉验证的切分方法对象
kfold = model_selection.KFold(n_splits=5, random_state=2018)

# 构建随机森林分类器
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_feature_num)

# 同上个例子
result = model_selection.cross_val_score(model, X, Y, cv=kfold)

# 同上个例子
print(result.mean())

0.7552754435107376


### Adaboost

In [14]:
# 串行的集成模型
from sklearn.ensemble import AdaBoostClassifier

# 反复“做题”25次
num_trees = 25

# 5折交叉验证的切分方法对象
kfold = model_selection.KFold(n_splits=5, random_state=2018)

# 构建adboost分类器
model = AdaBoostClassifier(n_estimators=num_trees, random_state=2018)

# 同上个例子
result = model_selection.cross_val_score(model, X, Y, cv=kfold)

# 同上个例子
print(result.mean())

0.7513623631270689
